In [ ]:
import os
import json 
import pandas as pd
import traceback


In [76]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

In [78]:
load_dotenv()

True

In [79]:
my_api = os.getenv("my_key")

In [80]:
llm = ChatGoogleGenerativeAI(model = "gemini-3-flash-preview" ,
                            google_api_key = my_api)

In [56]:
from langchain_core.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [57]:
Template = '''
text : {text}
You are a expert at creating multiple choice questions (MCQs) for educational purposes.
Based on the above text, generate {number} multiple choice questions (MCQs) for {subject} students in {tone} tone.
Make shure each question has 4 options (A, B, C, D) and indicate the correct answer also make shure that questions are not repeated in the output.
make shure the format you responce like responce_json below and use it as a guide:
## Responce_json
{responce_json}
'''
responce_json = {
    "1":{
        "question": "mcq",
        "options": {
            "A": "option1",
            "B": "option2",
            "C": "option3",
            "D": "option4"
        },
        "correct_answer": "correct_answer"
    },
    "2":{
        "question": "mcq",
        "options": {
            "A": "option1",
            "B": "option2",
            "C": "option3",
            "D": "option4"
        },
        "correct_answer": "correct_answer"
    }
}
prompt = PromptTemplate(input_variables=["text", "number", "tone", "responce_json"],
                        template=Template)
quiz_generation_chain = prompt|llm
Template2 = '''
Your are an expert english grammarian and writer . Give a multiple choice question quiz for {subject} student
you need to evluate the complexity of the questions and give the complete analysis of the quiz. onluy use at  max 50 words for complixty 
if the quiz is not at per the cognitive and analytical ablity of the student ,update the question of the quiz which need to be change and change the tone shuch that it prefectly fits the student ablity
Quize : {quiz}'''
prompt2 = PromptTemplate(input_variables=["subject", "quiz"],
                        template=Template2)
evaluation_chain = prompt2|llm|StrOutputParser()
final_chain = (
    
    
           # Run chain 1 to get the 'quiz' text
    RunnablePassthrough.assign(quiz=quiz_generation_chain)# Pass the 'subject' from the original input
    
    | evaluation_chain
)
with open(r"C:\Data Sceience\PROJECTS\mcqgen\data.txt", "r") as f:
    text = f.read()
Responce_json = json.dumps(responce_json)
resopnce = final_chain.invoke({"text": text, "number": 5, "tone": "intermidate", "subject": "machine learning", "responce_json": Responce_json})

In [62]:
from langchain_core.output_parsers import JsonOutputParser

# Define the parser
parser = JsonOutputParser()

# Update your chain to include the parser at the end
final_chain = (
    RunnablePassthrough.assign(quiz=quiz_generation_chain) 
    | evaluation_chain
    | parser  # <--- Add this! It automatically strips text and keeps JSON
)

# Run it
result = final_chain.invoke({
    "text": text, 
    "number": 5, 
    "tone": "intermediate", 
    "subject": "machine learning", 
    "responce_json": json.dumps(responce_json)
})

print(result) # This will be a clean Python Dictionary, no extra text!

{'1': {'question': "In Mitchell’s framework (E, T, P), if a model's performance (P) at task (T) improves with experience (E), how is 'Experience' typically quantified in modern supervised learning?", 'options': {'A': 'The complexity of the underlying algorithm', 'B': 'The volume of labeled training data processed', 'C': 'The number of hidden layers in a neural network', 'D': 'The reduction in computational latency'}, 'correct_answer': 'B'}, '2': {'question': "Which of the following scenarios best illustrates the 'Clustering' objective within Unsupervised Learning?", 'options': {'A': 'Predicting the continuous market value of a house based on square footage.', 'B': "Assigning emails to 'Spam' or 'Inbox' categories using historical labels.", 'C': 'Partitioning a customer database into distinct personas based on purchasing patterns without prior labels.', 'D': "Optimizing a robot's path through a maze via a reward-based feedback loop."}, 'correct_answer': 'C'}, '3': {'question': "When a m

In [63]:
result 

{'1': {'question': "In Mitchell’s framework (E, T, P), if a model's performance (P) at task (T) improves with experience (E), how is 'Experience' typically quantified in modern supervised learning?",
  'options': {'A': 'The complexity of the underlying algorithm',
   'B': 'The volume of labeled training data processed',
   'C': 'The number of hidden layers in a neural network',
   'D': 'The reduction in computational latency'},
  'correct_answer': 'B'},
 '2': {'question': "Which of the following scenarios best illustrates the 'Clustering' objective within Unsupervised Learning?",
  'options': {'A': 'Predicting the continuous market value of a house based on square footage.',
   'B': "Assigning emails to 'Spam' or 'Inbox' categories using historical labels.",
   'C': 'Partitioning a customer database into distinct personas based on purchasing patterns without prior labels.',
   'D': "Optimizing a robot's path through a maze via a reward-based feedback loop."},
  'correct_answer': 'C'},


In [73]:
import pandas as pd

# 1. Create an empty list to hold the cleaned rows
table_data = []

# 2. Loop through the dictionary to flatten the data
# quiz_data is the dictionary you extracted: {"1": {...}, "2": {...}}
for key, value in result.items():
    row = {
        "Question_ID": key,
        "Question": value.get("question"),
        "Option_A": value["options"].get("A"),
        "Option_B": value["options"].get("B"),
        "Option_C": value["options"].get("C"),
        "Option_D": value["options"].get("D"),
        "Correct_Answer": value.get("correct_answer")
    }
    table_data.append(row)

# 3. Convert list to DataFrame
quize = pd.DataFrame(table_data)

# 4. Display the result
# print(quize.head())

# Optional: Save to CSV
# quize.to_csv("my_quiz.csv", index=False)

In [72]:
quize

,Question_ID,Question,Option_A,Option_B,Option_C,Option_D,Correct_Answer
0,1,"In Mitchell’s framework (E, T, P), if a model'...",The complexity of the underlying algorithm,The volume of labeled training data processed,The number of hidden layers in a neural network,The reduction in computational latency,B
1,2,Which of the following scenarios best illustra...,Predicting the continuous market value of a ho...,Assigning emails to 'Spam' or 'Inbox' categori...,Partitioning a customer database into distinct...,Optimizing a robot's path through a maze via a...,C
2,3,When a machine learning model performs excepti...,Underfitting,Dimensionality reduction,Overfitting,Association rule learning,C
3,4,Generative Adversarial Networks (GANs) utilize...,One reduces dimensionality while the other inc...,One generates synthetic data while the other a...,One performs classification while the other pe...,One manages the input features while the other...,B
4,5,What is the fundamental difference between Sup...,Supervised learning requires a scalar reward s...,Supervised learning maps inputs to outputs bas...,"Supervised learning is only for discrete data,...",There is no functional difference; they are sy...,B


In [74]:
quize.to_csv("machine_learning_quiz.csv", index=False)